In [1]:
import requests
import nba_api
import pandas as pd
import numpy as np
from datetime import date
from nba_api.stats.endpoints import (playbyplayv2,playbyplay, leaguehustlestatsplayer,
                                     leaguedashptstats,leaguestandings,
                                     playerdashptreb,leaguedashplayerbiostats)
import time

In [2]:
pbp2023=pd.read_csv(filepath_or_buffer="Data/2022-23_pbp.csv")
pbp2023["next_home_play"]=pbp2023.HOMEDESCRIPTION.shift(-1)
pbp2023["next_visitor_play"]=pbp2023.VISITORDESCRIPTION.shift(-1)

# The Spark Plug Award (sponsored by Lt. Surge, presented by American Express CEO Stephen J Squeri)
Most charges drawn per 36 minutes (minimum 70% of games played), credit to morron88 for the idea to separate charges & loose balls in 2020

In [3]:
compact_standings=leaguestandings.LeagueStandings(league_id='00',
                                             season="2022-23",season_type="Regular Season").\
standings.get_data_frame()[['TeamID','TeamName','WINS','LOSSES']]
compact_standings['TeamGP']=compact_standings.WINS+compact_standings.LOSSES

hustle=leaguehustlestatsplayer.LeagueHustleStatsPlayer(
    per_mode_time="PerGame",season="2022-23",
    season_type_all_star="Regular Season").hustle_stats_player.get_data_frame()

In [4]:
hustle_w_gp_qualify=hustle.merge(compact_standings,how='left',left_on='TEAM_ID',right_on='TeamID')
hustle_w_gp_qualify['G_PERCENT']=hustle_w_gp_qualify.G/hustle_w_gp_qualify.TeamGP
hustle_70percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.7').copy()
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']]=\
hustle_70percent_gp[['CHARGES_DRAWN','LOOSE_BALLS_RECOVERED','DEFLECTIONS','SCREEN_AST_PTS']].\
div(hustle_70percent_gp.MIN,axis=0).multiply(36,axis=0)

In [5]:
hustle_70percent_gp.nlargest(10,columns='CHARGES_DRAWN',keep='all')[['PLAYER_NAME','CHARGES_DRAWN']]

,PLAYER_NAME,CHARGES_DRAWN
312,Kevin Love,0.954000
182,Isaiah Joe,0.508901
421,Patrick Beverley,0.411808
208,Jalen Brunson,0.411429
102,Davion Mitchell,0.377901
26,Austin Reaves,0.375000
491,Tim Hardaway Jr.,0.356436
177,Herbert Jones,0.352703
76,Clint Capela,0.338346
11,Alex Caruso,0.321702


Shoutout to Jaylin Williams, who played in only 60% of games, but drew 1.69 charges per 36 minutes!

# The Most Loose Balls Recovered Award (sponsored by Hungry Hungry Hippos, presented by Dennis Rodman & [Nene’s doctor](https://www.espn.com/nba/news/story?id=3197423))

Per 36 minutes, minimum 70% of games played

In [6]:
hustle_70percent_gp.nlargest(10,columns='LOOSE_BALLS_RECOVERED',keep='all')[['PLAYER_NAME','LOOSE_BALLS_RECOVERED']]

,PLAYER_NAME,LOOSE_BALLS_RECOVERED
475,T.J. McConnell,1.411765
479,Tari Eason,1.400000
425,Paul Reed,1.354128
263,Jordan Goodwin,1.132584
525,Wenyen Gabriel,1.096689
226,Jarred Vanderbilt,1.075519
273,Josh Hart,1.058824
132,Donovan Mitchell,1.045810
463,Shai Gilgeous-Alexander,1.044507
358,Markelle Fultz,1.033784


# The Plexiglass Award

most deflections per 36 minutes, minimum 70% of games played

In [7]:
hustle_70percent_gp.nlargest(10,columns='DEFLECTIONS',keep='all')[['PLAYER_NAME','DEFLECTIONS']]

,PLAYER_NAME,DEFLECTIONS
11,Alex Caruso,5.162553
363,Matisse Thybulle,4.558580
177,Herbert Jones,4.110811
475,T.J. McConnell,3.952941
108,De'Anthony Melton,3.922581
425,Paul Reed,3.765138
155,Fred VanVleet,3.727520
482,Terence Davis,3.654962
463,Shai Gilgeous-Alexander,3.650704
269,Jose Alvarado,3.650233


# The Wes Unseld Memorial Brick Wall Award

most points generated by screen assists per 36 minutes, minimum 70% of games played

In [8]:
hustle_70percent_gp.nlargest(10,columns='SCREEN_AST_PTS',keep='all')[['PLAYER_NAME','SCREEN_AST_PTS']]

,PLAYER_NAME,SCREEN_AST_PTS
315,Kevon Looney,14.279498
130,Domantas Sabonis,13.910983
409,Onyeka Okongwu,13.854545
138,Drew Eubanks,13.726108
447,Rudy Gobert,13.438436
142,Dwight Powell,13.218750
76,Clint Capela,13.073684
32,Bismack Biyombo,12.914685
345,Luke Kornet,12.492308
248,Jock Landale,12.447887


Shoutout to Steven Adams, who played in only 51% of games, but generated 16.83 points by screen assists per 36 minutes!

# The No Fly Zone Award (presented by Dikembe Mutumbo)*

most blocked dunks as the blocking player

In [9]:
misses=pbp2023[pbp2023.HOMEDESCRIPTION.str.startswith('MISS',na=False)| 
           pbp2023.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
missed_dunks=misses[misses.HOMEDESCRIPTION.str.contains('Dunk',na=False)|
                   misses.VISITORDESCRIPTION.str.contains('Dunk',na=False)]
blocked_dunks=missed_dunks[missed_dunks.HOMEDESCRIPTION.str.contains('BLOCK',na=False)|
                   missed_dunks.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
#as the blocker
blocked_dunks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
15,Brook Lopez,23
54,Drew Eubanks,20
87,Jaren Jackson Jr.,17
154,Myles Turner,17
199,Walker Kessler,15
89,Jarrett Allen,14
161,Nick Richards,14
32,Daniel Gafford,13
100,Joel Embiid,13
128,Kristaps Porzingis,13


# The Rejected for Boarding Award (sponsored by United Airlines)*

most blocked dunks as the dunking player (credit to Legdrop\_soup for the idea and asw7412 for the sponsor)

In [10]:
#as the blockee
blocked_dunks.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
0,Aaron Gordon,13
85,Ivica Zubac,11
210,Tari Eason,10
104,Jarrett Allen,9
116,Joel Embiid,8
134,Kenyon Martin Jr.,8
144,Kristaps Porzingis,8
190,Patrick Williams,8
228,Walker Kessler,8
9,Anthony Davis,7


# The No Time to Taunt Award (presented by Tim Duncan)*

highest percent of blocks that stayed inbounds & recovered by blocker's team (credit to gibberisle for the idea)

In [11]:
blocks=pbp2023[pbp2023.HOMEDESCRIPTION.str.contains('BLOCK',na=False)| 
           pbp2023.VISITORDESCRIPTION.str.contains('BLOCK',na=False)]
blocks['home_or_visitor_block']=np.where(blocks.HOMEDESCRIPTION.str.contains('BLOCK',na=False),"home","visitor")

team_rebounds=pbp2023[pbp2023.HOMEDESCRIPTION.str.endswith('Rebound',na=False)| 
           pbp2023.VISITORDESCRIPTION.str.endswith('Rebound',na=False)]

outer=blocks.merge(team_rebounds,how='outer',indicator=True,on=['GAME_ID','PERIOD','PCTIMESTRING'])
blocks_wo_team_rebs = outer[(outer._merge=='left_only')].drop('_merge', axis=1).drop(list(outer.filter(regex = '_y')), axis = 1)

/Users/sumitro/opt/anaconda3/lib/python3.8/site-packages/pandas/core/frame.py:3607: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._set_item(key, value)


In [12]:
def recover_after_block(df):
    if ((df.home_or_visitor_block=="home") & (pd.isnull(df.next_visitor_play_x))):
        return 'recovered by own team'
    elif ((df.home_or_visitor_block=="visitor") & (pd.isnull(df.next_home_play_x))):
        return 'recovered by own team'
    else:
        return 'recovered by other team'

In [13]:
blocks_wo_team_rebs['recovered']=blocks_wo_team_rebs.apply(recover_after_block,axis=1)

player_blocks=blocks.groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'blocks'})

player_blocks_wo_team_rebs=pd.DataFrame(
    blocks_wo_team_rebs.groupby(['PLAYER3_NAME_x','recovered'])
    .size().reset_index().pivot_table(index='PLAYER3_NAME_x',columns='recovered',fill_value=0).to_records()
).rename(columns={"(0, 'recovered by other team')":'blocks_recovered_by_opp',
                 "(0, 'recovered by own team')":'blocks_recovered_by_own'})
player_blk_info=player_blocks.merge(player_blocks_wo_team_rebs,left_on=['PLAYER3_NAME'],right_on='PLAYER3_NAME_x')
player_blk_info["percent_blk_recovered_by_own"]=player_blk_info["blocks_recovered_by_own"]/player_blk_info["blocks"]
player_blk_info.query("blocks>=40").nlargest(10,columns='percent_blk_recovered_by_own',keep='all')

,PLAYER3_NAME,blocks,PLAYER3_NAME_x,blocks_recovered_by_opp,blocks_recovered_by_own,percent_blk_recovered_by_own
336,Mo Bamba,43,Mo Bamba,6,31,0.720930
174,Jabari Smith Jr.,74,Jabari Smith Jr.,11,52,0.702703
121,Draymond Green,60,Draymond Green,9,42,0.700000
374,Patrick Williams,70,Patrick Williams,14,48,0.685714
425,Tari Eason,47,Tari Eason,8,32,0.680851
310,Luke Kornet,46,Luke Kornet,11,31,0.673913
63,Christian Koloko,57,Christian Koloko,13,38,0.666667
358,O.G. Anunoby,50,O.G. Anunoby,6,33,0.660000
159,Isaiah Hartenstein,64,Isaiah Hartenstein,14,42,0.656250
6,Al Horford,61,Al Horford,12,40,0.655738


# The “Oops, I Dunked It Again” Award (sponsored by Britney Spears, presented by Gary Payton & Shawn Kemp)*

Most prolific alley-oop duo (credit to lactardenthusiast for the idea)

In [14]:
#get alley oops
alley_oops=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Alley',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('Alley',na=False)]
#remove missed alley oop attempts
made_alley_oops=alley_oops[~alley_oops.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~alley_oops.VISITORDESCRIPTION.str.startswith('MISS',na=False)]
#remove alley oops that have missing 2nd player (self-alley oop or perhaps missing data?)
made_alley_oops_complete=made_alley_oops[~made_alley_oops.PLAYER2_NAME.isna()].copy()
made_alley_oops_complete['p1']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].min(axis=1)
made_alley_oops_complete['p2']=made_alley_oops_complete[['PLAYER1_NAME','PLAYER2_NAME']].max(axis=1)
made_alley_oops_complete.groupby(['p1','p2']).size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,p1,p2,count
228,Clint Capela,Trae Young,91
826,Onyeka Okongwu,Trae Young,35
632,John Collins,Trae Young,33
396,Dwight Powell,Luka Doncic,30
130,Brandon Clarke,Ja Morant,25
12,Aaron Gordon,Nikola Jokic,24
807,Nic Claxton,Spencer Dinwiddie,21
283,Darius Garland,Evan Mobley,20
494,Ja Morant,Jaren Jackson Jr.,20
728,Kyle Anderson,Rudy Gobert,20


# The “He Trick Y’All, Running Around, Doing Nothing” Award (sponsored by Russell Westbrook, presented by Tony Snell)*

Lowest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played)

In [15]:
hustle_50percent_gp=hustle_w_gp_qualify.query('G_PERCENT >= 0.5').copy()
#traditional defensive stats approximate by tracking
#(deflections ~ steals, contested shots ~ blocks, def reb ~ boxouts)
per_36_percent_ranks=hustle_50percent_gp[['CHARGES_DRAWN', 'CONTESTED_SHOTS_2PT','CONTESTED_SHOTS_3PT', 'DEFLECTIONS', 
               'DEF_BOXOUTS','DEF_LOOSE_BALLS_RECOVERED']].\
div(hustle_50percent_gp.MIN,axis=0).multiply(36,axis=0).apply(lambda x: x.rank(pct=True)).add_suffix("_pct_rank")
per_36_percent_ranks["sum"]=per_36_percent_ranks.sum(axis=1)
hustle_50percent_gp_ranks=hustle_50percent_gp.merge(per_36_percent_ranks,how='left',left_index=True,right_index=True)

In [16]:
hustle_50percent_gp_ranks.to_csv(path_or_buf="Output Data/Hustle Ranks.csv")

In [17]:
hustle_50percent_gp_ranks.nsmallest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
146,Eric Gordon,0.171920,0.008596,0.008596,0.386819,0.068768,0.065903,0.710602
82,Collin Sexton,0.461318,0.186246,0.080229,0.183381,0.014327,0.051576,0.977077
210,Jalen Green,0.171920,0.057307,0.432665,0.127507,0.005731,0.249284,1.044413
320,Klay Thompson,0.171920,0.338109,0.097421,0.286533,0.060172,0.312321,1.266476
348,Malaki Branham,0.171920,0.200573,0.297994,0.083095,0.008596,0.570201,1.332378
435,Reggie Bullock,0.171920,0.126074,0.037249,0.263610,0.495702,0.275072,1.369628
461,Seth Curry,0.171920,0.054441,0.191977,0.627507,0.143266,0.194842,1.383954
236,Jaylen Nowell,0.171920,0.111748,0.054441,0.702006,0.106017,0.272206,1.418338
136,Doug McDermott,0.171920,0.375358,0.212034,0.002865,0.277937,0.395415,1.435530
91,Damion Lee,0.171920,0.134670,0.455587,0.034384,0.424069,0.217765,1.438395


In [18]:
hustle_50percent_gp_ranks[hustle_50percent_gp_ranks['PLAYER_NAME'].isin(['Patrick Beverley','Russell Westbrook'])].filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
421,Patrick Beverley,0.977077,0.197708,0.335244,0.765043,0.134670,0.361032,2.770774
449,Russell Westbrook,0.716332,0.180516,0.323782,0.661891,0.234957,0.707736,2.825215


# The "Got that Dawg in Him" Award (presented by Air Bud)*

Highest sum of per-36 percentile ranks in the following: charges, contested shots, deflections, defensive boxouts, defensive loose balls recovered (minimum 50% of games played) (credit to memeticengineering for the idea)

In [19]:
hustle_50percent_gp_ranks.nlargest(n=10,columns='sum',keep='all').filter(regex='PLAYER_NAME|sum|pct_rank$',axis=1)

,PLAYER_NAME,CHARGES_DRAWN_pct_rank,CONTESTED_SHOTS_2PT_pct_rank,CONTESTED_SHOTS_3PT_pct_rank,DEFLECTIONS_pct_rank,DEF_BOXOUTS_pct_rank,DEF_LOOSE_BALLS_RECOVERED_pct_rank,sum
425,Paul Reed,0.495702,0.962751,0.931232,0.974212,0.885387,0.977077,5.226361
486,Thaddeus Young,0.936963,0.702006,0.802292,0.994269,0.681948,0.885387,5.002865
11,Alex Caruso,0.957020,0.578797,0.988539,0.997135,0.475645,0.945559,4.942693
177,Herbert Jones,0.962751,0.613181,0.876791,0.985673,0.550143,0.899713,4.888252
17,Andre Drummond,0.704871,0.845272,0.656160,0.925501,0.876791,0.833811,4.842407
237,Jaylin Williams,1.000000,0.882521,0.664756,0.418338,0.997135,0.787966,4.750716
137,Draymond Green,0.888252,0.848138,0.928367,0.891117,0.911175,0.242120,4.709169
185,Isaiah Roby,0.845272,0.756447,0.885387,0.621777,0.570201,0.931232,4.610315
384,Moritz Wagner,0.982808,0.730659,0.510029,0.696275,0.888252,0.790831,4.598854
33,Blake Griffin,0.997135,0.773639,0.962751,0.593123,0.962751,0.266476,4.555874


# The Bowling Ball Award (sponsored by [Pete Weber](https://www.youtube.com/watch?v=gKQOXYB2cd8), presented by Glen "Big Baby" Davis)*

most charges committed (credit to Kdog122025 for the idea)

In [20]:
charges=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Charge',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('Charge',na=False)]

charges.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
104,Giannis Antetokounmpo,23
209,Kyle Kuzma,19
18,Bennedict Mathurin,16
69,Deni Avdija,15
182,Julius Randle,15
12,Anthony Edwards,14
247,O.G. Anunoby,14
254,Paolo Banchero,14
16,Bam Adebayo,13
158,Jerami Grant,12


# "The Good Ol' Hockey Game, is the Best Game You Can Name" Award (presented by Dominik Hasek)*

most goaltends committed (credit to Kdog122025 for the idea)

In [21]:
goaltends=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Goaltending',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('Goaltending',na=False)]
goaltends.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
39,Daniel Gafford,19
171,Nic Claxton,19
172,Nick Richards,16
87,Jalen Duren,13
153,Mark Williams,13
94,Jaren Jackson Jr.,12
194,Rudy Gobert,12
36,Clint Capela,11
164,Mitchell Robinson,11
167,Myles Turner,11


# "The Thing about Arsenal Is They Always Try to Walk It In" Award (presented by MLS Commissioner Don Garber)*

most kicked ball violations

In [22]:
kicked_balls=pbp2023[pbp2023.HOMEDESCRIPTION.str.contains('Kick',na=False)|
                     pbp2023.VISITORDESCRIPTION.str.contains('Kick',na=False)]
kicked_balls.groupby('PLAYER1_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME,count
248,Nikola Jokic,47
244,Nic Claxton,19
249,Nikola Vucevic,19
131,Jakob Poeltl,18
128,Jaden McDaniels,16
81,Domantas Sabonis,15
180,Jusuf Nurkic,13
64,De'Aaron Fox,11
162,Jonas Valanciunas,11
208,Luka Doncic,11


# The "David vs Goliath" Award (presented by Dwyane Wade)*

most shots blocked with at least a 5-inch difference in height

In [23]:
blocks=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('BLOCK',na=False)|
                          pbp2023['VISITORDESCRIPTION'].str.contains('BLOCK',na=False)]

player_bio=leaguedashplayerbiostats.LeagueDashPlayerBioStats().league_dash_player_bio_stats.get_data_frame()

blocking_player_heights=blocks.merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER1_ID',right_on='PLAYER_ID').merge(player_bio[['PLAYER_ID','PLAYER_HEIGHT_INCHES']],left_on='PLAYER3_ID',right_on='PLAYER_ID')
blocking_player_heights["height_difference"]=blocking_player_heights['PLAYER_HEIGHT_INCHES_x']-blocking_player_heights['PLAYER_HEIGHT_INCHES_y']
blocking_player_heights.query('height_difference >= 5').groupby('PLAYER3_NAME').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER3_NAME,count
78,Fred VanVleet,28
211,Patrick Beverley,23
170,Kyrie Irving,21
33,Chris Paul,18
69,Donovan Mitchell,16
169,Kyle Lowry,16
24,CJ McCollum,15
44,Damian Lillard,12
54,De'Anthony Melton,11
140,Josh Okogie,11


# The Most 3-Pt Shooting Fouls Committed Award*

credit to watchingsongsDL, kingcobweb & An-Indian-In-The-NBA for the idea

In [24]:
third_free_throws=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)|
                          pbp2023['VISITORDESCRIPTION'].str.contains('Free Throw 3 of 3',na=False)]

shooting_fouls=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('S.FOUL',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('S.FOUL',na=False)]

fouls_on_missed_threes=shooting_fouls.merge(right=third_free_throws,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

three_pointers=pbp2023[pbp2023['HOMEDESCRIPTION'].str.contains('3PT',na=False)|
        pbp2023['VISITORDESCRIPTION'].str.contains('3PT',na=False)]

made_threes=three_pointers[~three_pointers.HOMEDESCRIPTION.str.startswith('MISS',na=False) & 
           ~three_pointers.VISITORDESCRIPTION.str.startswith('MISS',na=False)]

fouls_on_made_threes=shooting_fouls.merge(right=made_threes,how='inner',on=['GAME_ID','PERIOD','PCTIMESTRING'])

fouls_on_threes=fouls_on_missed_threes.append(fouls_on_made_threes)
fouls_on_threes.groupby('PLAYER1_NAME_x').size().reset_index().rename(columns={0:'count'}).nlargest(10,columns='count',keep='all')

,PLAYER1_NAME_x,count
89,Dillon Brooks,10
74,De'Anthony Melton,9
80,Deni Avdija,9
149,Jalen McDaniels,9
227,Luguentz Dort,9
52,Coby White,8
73,De'Andre Hunter,8
103,Evan Mobley,8
200,Keegan Murray,8
323,Trae Young,8


# The “Fine, I’ll Do It Myself” Award (sponsored by Thanos, presented by Allen Iverson)

Highest percentage of unassisted field goals, minimum 50% of games played (https://www.nba.com/stats/players/scoring/?sort=GP&dir=-1)

1. Luka Doncic (84.7%)
2. Shai Gilgeous-Alexander (79%)
3. Trae Young (78.7%)
4. Ja Morant & De'Aaron Fox (76.1%)

# The “You Gotta Feed Me” Award (presented by Joey Chestnut & Marcin Gortat)

Highest percentage of assisted field goals, minimum 50% of games played

1. Duncan Robinson (97.7%)
2. Davis Bertans (97.1%)
3. Reggie Bullock (95.3%)
4. Joe Harris (94%)
5. Mike Muscala (93.9%)

# The “FUCK OUTTA HERE, I GOT THAT SHIT” Award (presented by Carmelo Anthony)

Lowest contested rebound percentage, minimum 50% of games played

In [25]:
rebounding=playerdashptreb.PlayerDashPtReb(team_id=0,player_id=0).overall_rebounding.get_data_frame()

games_percentages=hustle_w_gp_qualify.copy()[['PLAYER_ID','PLAYER_NAME','G_PERCENT']]

reb_w_gp_qualify=rebounding.merge(games_percentages,left_on='PLAYER_ID',right_on='PLAYER_ID').query('G_PERCENT >= 0.5')

reb_w_gp_qualify.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
122,Tyus Jones,0.075
94,Seth Curry,0.080
76,Dennis Schroder,0.091
132,Cameron Payne,0.094
63,Austin Rivers,0.096
15,Eric Gordon,0.104
255,Trae Young,0.106
475,Malaki Branham,0.110
336,Tyrese Haliburton,0.112
345,Tyrese Maxey,0.114


alternatively: restricting to players > 6 foot 6 inches in height

In [26]:
above_66=player_bio.query('PLAYER_HEIGHT_INCHES > 6*12+6')

above_66.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nsmallest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
78,Joe Ingles,0.133
153,Terrence Ross,0.136
3,Amir Coffey,0.170
12,Bojan Bogdanovic,0.176
70,Jayson Tatum,0.180
45,Gordon Hayward,0.186
96,Kevin Huerter,0.187
135,Paul George,0.187
43,Georges Niang,0.189
42,Franz Wagner,0.198


# The "Glass Cleaner" Award (presented by Dennis Rodman, sponsored by Windex)

Highest contested rebound percentage, minimum 50% of games played (https://www.nba.com/stats/players/rebounding?PerMode=Totals&dir=D&sort=REB_CONTEST_PCT)

In [27]:
reb_w_gp_qualify.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME','C_REB_PCT']]

,PLAYER_NAME,C_REB_PCT
265,Wenyen Gabriel,0.637
245,Mitchell Robinson,0.605
294,Romeo Langford,0.582
211,Luke Kornet,0.579
409,Isaiah Jackson,0.560
413,Day'Ron Sharpe,0.555
158,Jakob Poeltl,0.549
57,Anthony Davis,0.541
495,Christian Koloko,0.532
31,Taj Gibson,0.527


alternatively: restricting to players < 6 foot 7 inches in height

In [28]:
below_67=player_bio.query('PLAYER_HEIGHT_INCHES < 6*12+7')
below_67.merge(reb_w_gp_qualify,left_on='PLAYER_ID',right_on='PLAYER_ID')\
.nlargest(n=10,columns='C_REB_PCT',keep='all')[['PLAYER_NAME_x','C_REB_PCT']]

,PLAYER_NAME_x,C_REB_PCT
149,Romeo Langford,0.582
49,Derrick Jones Jr.,0.507
100,Josh Okogie,0.446
108,Kenrich Williams,0.425
3,Aaron Wiggins,0.395
118,Luguentz Dort,0.374
78,Ish Wainright,0.371
58,Draymond Green,0.366
9,Anthony Lamb,0.362
69,Grant Williams,0.337


# The Rotation Awards

I value depth over one solitary star (it’s a rotation after all). My attempt to make this more objective than past years:
- Get position estimates from Basketball-Reference
    - players with most minutes at PG & SG: guard
    - players with most minutes at SG & SF or SF & PF: wing
    - players with most minutes at PC & C: big
    - exceptions: all players greater than 7 foot are bigs and all players less than 6 foot are guards
- Filter out players with less than 12 MP/G
- Rank teams based on highest positional VORP without positional VORP leader

# The Best Guard Rotation Award (sponsored by Buckingham Palace)

East
1. Wizards (Delon Wright, Bradley Beal, Monte Morris, Jordan Goodwin, Kendrick Nunn)
2. 76ers (James Harden, De’Anthony Melton, Tyrese Maxey)
3. Cavaliers (Donovan Mitchell, Darius Garland, Ricky Rubio)
4. Boston (Derrick White, Malcolm Brogdon, Marcus Smart, Payton Pritchard)
5. Knicks (Jalen Brunson, Immanuel Quickley, Derrick Rose)

West
1. Grizzlies (Ja Morant, Tyus Jones, Luke Kennard)
2. Mavericks (Luka Doncic, Spencer Dinwiddie, Kyrie Irving, McKinley Wright IV, Frank Ntilikina)
3. Warriors (Steph Curry, Donte DiVincenzo, Ty Jerome, Jordan Poole, Gary Payton II)
4. Nuggets (Jamal Murray, Bruce Brown, Bones Hyland, Reggie Jackson)
5. Pelicans (CJ McCollum, Devonte’ Graham, Jose Alvarado, Dyson Daniels)

# The Best Wing Rotation Award (co-sponsored by Lou Williams and Magic City)

East
1. Boston (Jayson Tatum, Jaylen Brown, Sam Hauser, Grant Williams)
2. Bulls (Zach LaVine, DeMar DeRozan, Derrick Jones Jr, Ayo Dosunmu)
3. Raptors (Scottie Barnes, Chris Boucher, OG Anunoby, Thad Young, Juancho Hernangomez)
4. Cavaliers (Caris LeVert, Isaac Okoro, Dean Wade, Cedi Osman, Lamar Stevens)
5. Pistons (Bojan Bogdanovic, Alec Burks, Saddiq Bey, Hamidou Diallo, Kevin Knox, Eugene Omoruyi, Isaiah Livers, Rodney McGruder, Jared Rhoden)

West
1. Suns (Devin Booker, Mikal Bridges, Cam Johnson, Torrey Craig, Josh Okogie, Ish Wainright, Damion Lee, Terrence Ross, TJ Warren)
2. Clippers (Kawhi Leonard, Paul George, Terance Mann, Robert Covington, Marcus Morris, Luke Kennard, Amir Coffey)
3. Kings (Kevin Huerter, Harrison Barnes, Keegan Murray, Malik Monk, Terence Davis, Kessler Edwards)
4. Pelicans (Trey Murphy III, Brandon Ingram, Herb Jones, Josh Richardson, Naji Marshall)
5. Timberwolves (Kyle Anderson, Anthony Edwards, Jaden McDaniels, Nickeil Alexander-Walker, Taurean Prince, Jaylen Nowell)

# The Best Big Rotation Award (jointly sponsored by Tom Hanks, Cadbury and Sex and the City)

East
1. Bucks (Giannis Antetokounmpo, Brook Lopez, Bobby Portis)
2. Nets (Kevin Durant, Nic Claxton, Cam Johnson)
3. Knicks (Julius Randle, Mitchell Robinson, Isaiah Hartenstein, Jericho Sims, Obi Toppin)
4. Cavaliers (Jarrett Allen, Evan Mobley, Kevin Love)
5. Hawks (Clint Capela, Onyeka Okongwu, Jalen Johnson, John Collins)

West
1. Lakers (LeBron James, Anthony Davis, Thomas Bryant, Wenyen Gabriel)
2. Grizzlies (Jaren Jackson Jr, Brandon Clarke, Santi Aldama, Steven Adams, Xavier Tillman)
3. Jazz (Lauri Markkanen, Walker Kessler, Jarred Vanderbilt, Kelly Olynyk, Damian Jones, Rudy Gay)
4. Pelicans (Zion Williamson, Larry Nance Jr, Jonas Valanciunas, Willy Hernangomez, Jaxson Hayes)
5. Timberwolves (Rudy Gobert, Naz Reid, Karl-Anthony Towns)